In [53]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem.MolStandardize import rdMolStandardize
from mordred import Calculator, descriptors
from openbabel import pybel
from tqdm import tqdm
import os
import numpy as np

In [54]:
# compound_name = "1799A-18F10"
# compound_density_file = f"./Data/{compound_name}.xlsx"
# compound_mol_file = f"./Data/{compound_name}.mol"
# #mol_file2 = f"./Data/1781-18FPF-9811.mol"
output_file = f"./Result/output.csv"

In [55]:
# mols = []
# mol = Chem.MolFromMolFile(compound_mol_file)
# # 转换并输出SMILES
# mols.append(mol)

# mol = Chem.MolFromMolFile(mol_file2)
# mols.append(mol)


"""
    isomericSmiles: 异构体smiles, 暂时不清楚在什么情况下使用
    kekuleSmiles: kekulize一个分子, 表现为smiles里的C原子之间用=号连接, 需要在转换前调用Chem.Kekulize()进行处理
"""
# Chem.Kekulize(mol)
# smiles = Chem.MolToSmiles(mol, isomericSmiles=False, kekuleSmiles=True)
# print(smiles)
'''
original:
    Cc1c(SCc2ccc(CCCCF)cc2)oc2ccccc2c1=O
kekulized:
    CC1=C(SCC2=CC=C(CCCCF)C=C2)OC2=CC=CC=C2C1=O
''' 
# Chem.AllChem.Compute2DCoords(mol)


'\noriginal:\n    Cc1c(SCc2ccc(CCCCF)cc2)oc2ccccc2c1=O\nkekulized:\n    CC1=C(SCC2=CC=C(CCCCF)C=C2)OC2=CC=CC=C2C1=O\n'

In [56]:
def calculate_Mordred_descriptors(mols_files, ignore_3D=True, isomericSmiles=False, kekuleSmiles=True, output_file=None) -> pd.DataFrame:
    """
        计算输入的Mol格式化合物的分子描述符并保存到指定文件中

        params:
            mols_files:Mol格式文件可遍历列表
            output_file:输出的文件
            ignore_3D: True时只计算1D&2D描述符
            isomericSmiles:异构体SMILES
            kekuleSmiles:是否kekulize SMILES
    """
    smiles_list = []
    mols = []
    compound_id = []
    for file in mols_files:
        # 每个mol文件对应一种化合物id，将id记录保存
        filename = os.path.split(file)[-1]
        id = os.path.splitext(filename)[0]
        # TODO: 根据id读取化合物的实验浓度数据，获取血脑浓度数据并记录保存
        compound_id.append(id)

        # 转换成RDKit的Mol格式数据
        mol = Chem.MolFromMolFile(file)
        mols.append(mol)

        # 转换成SMILES列表
        if kekuleSmiles:
            Chem.Kekulize(mol)
        smiles = Chem.MolToSmiles(mol, isomericSmiles=isomericSmiles, kekuleSmiles=kekuleSmiles)
        smiles_list.append(smiles)
    
    # 创建Mordred特征计算器
    calc = Calculator(descriptors, ignore_3D=ignore_3D)

    # 计算特征
    all_mord = calc.pandas(mols).astype('float64')
    all_mord.insert(0, 'ID', compound_id)
    all_mord.insert(1, 'SMILES', smiles_list)
    
    # 数据导出
    if output_file is not None:
        output_suffix = os.path.splitext(output_file)[-1][1:]
        if output_suffix == 'csv':
            all_mord.to_csv(output_file, index=False, float_format="%.6f")
        elif output_suffix == 'xlsx':
            #TODO
            pass
        else:
            print("Wrong file format")
    return all_mord

In [57]:
def calculate_Pybel_descriptors(mols_files, output_file=None, isomericSmiles=False, kekuleSmiles=True):
    smiles_list = []
    mols = []
    compound_id = []
    for file in mols_files:
        # 每个mol文件对应一种化合物id，将id记录保存
        filename = os.path.split(file)[-1]
        id = os.path.splitext(filename)[0]
        # TODO: 根据id读取化合物的实验浓度数据，获取血脑浓度数据并记录保存
        compound_id.append(id)

        res = pybel.readfile("mol", file)
        # res是迭代器，调用next来获取单个化合物数据(pybel.Molecule object)
        mols.append(res.__next__())

        mol = Chem.MolFromMolFile(file)
        # 转换成SMILES列表
        if kekuleSmiles:
            Chem.Kekulize(mol)
        smiles = Chem.MolToSmiles(mol, isomericSmiles=isomericSmiles, kekuleSmiles=kekuleSmiles)
        smiles_list.append(smiles)

    # 根据pybel的特征作为列名生成DataFrame
    full_data = pd.DataFrame(columns=pybel.descs, dtype=float)

    for mol in tqdm(mols):
        # 计算描述符
        descvalues = mol.calcdesc()
        # 得到的描述符更新到mol的数据中
        mol.data.update(descvalues)
        # 将mol的数据转换为DataFrame
        mol_data = pd.DataFrame.from_dict([dict(mol.data)])
        full_data = full_data.append(mol_data)
    # full_data.insert(0, 'SMILE', SMILES)
    # full_data = full_data.drop(axis=1,
    #                            columns=['cansmi', 'cansmiNS', 'formula', 'InChI',
    #                                     'InChIKey', 's', 'smarts', 'title', 'L5',
    #                                     'MW', 'nF', 'HBD', 'atoms', 'abonds', 'bonds', 'dbonds'])
    # full_data.to_csv(dst_file, index=False, float_format="%.6f")
    return full_data

In [58]:
def calculate_descriptors(mols_files, ignore_3D=True, isomericSmiles=False, kekuleSmiles=True, output_file=None) -> pd.DataFrame:
    """
        计算输入的Mol格式文件中化合物的分子描述符(Mordred & Pybel)并保存到指定文件中

        params:
            mols_files:Mol格式文件可遍历列表
            output_file:输出的文件
            ignore_3D: True时只计算1D&2D描述符
            isomericSmiles:异构体SMILES
            kekuleSmiles:是否kekulize SMILES
    """
    smiles_list = []
    mordred_mols = []
    pybel_mols = []
    compound_id = []
    
    for file in mols_files:
        try:
            # 每个mol文件对应一种化合物id，将id记录保存
            filename = os.path.split(file)[-1]
            id = os.path.splitext(filename)[0]

            # 转换成RDKit的Mol格式数据
            mol = Chem.MolFromMolFile(file)
            # 在rdkit中，分子在默认情况下是不显示氢的，但氢原子对于真实的几何构象计算有很大的影响，所以在计算3D构象前，需要使用Chem.AddHs()方法加上氢原子 
            # https://blog.csdn.net/dreadlesss/article/details/105613264
            mol_3d = Chem.AddHs(mol)
            # 使用距离几何算法初始化3D坐标
            Chem.AllChem.EmbedMolecule(mol_3d)
            # 使用MMFF94等力场进行优化。不过需要注意的是，MMFF力场中的原子类型编码采用了自身的芳香性模型，因此在使用MMFF相关方法后，分子的芳香属性（aromaticity flags）会改变
            Chem.AllChem.MMFFOptimizeMolecule(mol_3d)
            mordred_mols.append(mol_3d)

            res = pybel.readfile("mol", file)
            # res是迭代器，调用next来获取单个化合物数据(pybel.Molecule object)
            pybel_mols.append(res.__next__())

            # 转换成SMILES列表
            if kekuleSmiles:
                Chem.Kekulize(mol)
            smiles = Chem.MolToSmiles(mol, isomericSmiles=isomericSmiles, kekuleSmiles=kekuleSmiles)
            smiles_list.append(smiles)
            # TODO: 根据id读取化合物的实验浓度数据，获取血脑浓度数据并记录保存
            compound_id.append(id)
        except Exception as e:
            print(e)
            if 'Conformer Id' in str(e):
                print(file)
            continue


    """计算Mordred描述符"""
    # 创建Mordred特征计算器
    calc = Calculator(descriptors, ignore_3D=ignore_3D)

    # 计算特征
    mordred_desc = calc.pandas(mordred_mols)

    mordred_desc = mordred_desc.replace('TRUE', value=1)
    mordred_desc = mordred_desc.replace('FALSE', value=0)
    mordred_desc = mordred_desc.astype('float64')
    # print(mordred_desc)
    mordred_desc.insert(0, 'ID', compound_id)
    mordred_desc.insert(1, 'SMILES', smiles_list)


    """计算Pybel描述符"""
    # 根据pybel的特征作为列名生成DataFrame
    pybel_desc = pd.DataFrame(columns=pybel.descs, dtype=float)

    for mol in tqdm(pybel_mols, desc='Calculating Pybel Descriptors: '):
        # 计算描述符
        descvalues = mol.calcdesc()
        # 得到的描述符更新到mol的数据中
        mol.data.update(descvalues)
        # 将mol的数据转换为DataFrame
        mol_data = pd.DataFrame.from_dict([dict(mol.data)])
        pybel_desc = pybel_desc.append(mol_data)
    pybel_desc.insert(0, 'ID', compound_id)
    pybel_desc = pybel_desc.drop(axis=1, columns=['nF', 'TPSA', 'MW', 'smarts', 's', 'title', 'MOL Chiral Flag',
                                                  'OpenBabel Symmetry Classes', 'InChI',
                                                  'InChIKey', 'L5', 'cansmi', 'cansmiNS', 'formula'])

    """合并二者特征"""
    if len(mordred_desc) > len(pybel_desc):
        how = 'right'
    else:
        how = 'left'
    combined_data = mordred_desc.set_index('ID').join(pybel_desc.set_index('ID'), on='ID', how=how)

    """导出数据"""
    if output_file is not None:
        output_suffix = os.path.splitext(output_file)[-1][1:]
        if output_suffix == 'csv':
            combined_data.to_csv(output_file, float_format="%.6f")
        elif output_suffix == 'xlsx':
            #TODO
            pass
        else:
            print("Wrong file format")
    return mordred_desc, pybel_desc

In [59]:
mol_files = []
data_list = os.listdir("./Data2")
for file in data_list:
    if file.endswith(".mol"):
        mol_files.append(f"./Data2/{file}")
print(len(mol_files))

# calculate_Mordred_descriptors(mol_files, ignore_3D=False, kekuleSmiles=False, output_file=output_file)
# calculate_Pybel_descriptors(mol_files, output_file)
calculate_descriptors(mol_files, ignore_3D=False, kekuleSmiles=False, output_file=output_file)

851


[17:38:01] UFFTYPER: Unrecognized charge state for atom: 7


Bad input file ./Data2/1069A-16β-F-DHT;18F-2.mol
Bad input file ./Data2/1069A-16β-F-Mib;18F-6.mol
Bad input file ./Data2/1069A-16β-F-MNT-18F-9.mol
Bad input file ./Data2/1069A-16β-F-MNT;18F-7.mol
Bad input file ./Data2/1069A-16β-F-T;18F-4.mol
Bad Conformer Id
./Data2/1085-18F8b.mol
Bad Conformer Id
./Data2/1085-18F8c.mol


KeyboardInterrupt: 

In [ ]:
def merge_desc_file(pybel_file, Mordred_file, output_file):
    pybel_data = pd.read_csv(pybel_file)
    Mordred_data = pd.read_csv(Mordred_file)

    SMILES = Mordred_data['SMILE']
    # 去除结果中的字符串内容
    Mordred_data = Mordred_data.replace(regex=['^[a-z].+'], value='')
    Mordred_data = Mordred_data.replace('TRUE', value=1)
    Mordred_data = Mordred_data.replace('FALSE', value=0)
    
    # 计算MWHBN: ((HBA + HBD)/sqrt(MW))
    HBD = Mordred_data['nHBDon']
    HBA = Mordred_data['nHBAcc']
    MW = Mordred_data['MW']

    HBN = HBD + HBA
    MW = MW.apply(np.sqrt)
    MWHBN = HBN / MW
    
    how = ''
    if len(Mordred_data) > len(pybel_data):
        how = 'right'
    else:
        how = 'left'
    Mordred_data['SMILE'] = SMILES
    combined_data = Mordred_data.set_index('SMILE').join(pybel_data.set_index('SMILE'), on='SMILE', how=how)
    # combined_data = Mordred_data.join(pybel_data, how=how)
    combined_data.insert(combined_data.shape[1], 'MWHBN', MWHBN.values) # 这里的MWHBN若不添加values转变为narray，会在插入时检查自身index与目标df的index，因为index不匹配导致插入NaN
    # combined_data['MWHBN'] = MWHBN.values
    combined_data.to_csv(output_file)